In [ ]:
import os
WEBHOOK_URL = os.environ['SLACK_WEBHOOK_URL']
CHANNEL_ID = os.environ['SLACK_CHANNEL_ID']

In [ ]:
def send_slack(message, text, color='good'):
    if color == 'good':
        message = f':grinning: {message}'
    if color == 'danger':
        message = f':slightly_frowning_face: {message}'

    import requests
    requests.post(
        WEBHOOK_URL,
        json={
            'channel': CHANNEL_ID,
            'username': 'Trading Notifications',
            'icon_url': 'https://ethereum.org/favicon-32x32.png',
            'text': message,
            'mrkdwn': True,
            'attachments': [{
                'title': '→ Trading Operations',
                'title_link': 'https://jhub.name/user/sheng_wei/lab',
                'text': text,
                'color': color,
            }]
        }
    )

# A Trader Stabilize Fiat Value of Crypto Currency

In [ ]:
TEST_RATIO = 1

In [ ]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 60
SAD_COOLING_TIME = 60 * 6
TRADE_RATE = 10
NOTIFY_RATE = 60
TRADE_AMOUNT = 0.001
MIN_UNIT_CC_INIT_TRADE_JPY = 26500
MAX_UNIT_CC_INIT_TRADE_JPY = MIN_UNIT_CC_INIT_TRADE_JPY << 10
MAX_SELL_COUNT = 1 << 10
MAX_USED_JPY = 800000
MAX_UNIT_CC_GAIN_JPY = 256
MAX_UNIT_CC_LOSS_JPY = MAX_UNIT_CC_INIT_TRADE_JPY
MIN_UNIT_CC_LOSS_JPY = 375 # TODO: Make it Adaptive
MAX_GAIN_JPY = 512 # useless
MAX_TOTAL_LOSS_JPY = 400000 << 10

In [ ]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [ ]:
pub = python_bitbankcc.public()

## Simulated One

In [ ]:
from unittest.mock import Mock
prv = Mock()

## Actual One

In [ ]:
# import os
# prv = python_bitbankcc.private(
#     os.environ['BITBANK_API_KEY'],
#     os.environ['BITBANK_API_SECRET']
# )

## Trading Operations

In [ ]:
def sell_all():
    now_ticker = pub.get_ticker('eth_jpy')
    now_sell_jpy = int(now_ticker.get('buy', '0'))
    now_buy_jpy = int(now_ticker.get('sell', '1'))

    please_sell_amount = TRADE_AMOUNT * bought_unit_amount
    prv.order(
        pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
    )

    bought_amount = TRADE_AMOUNT * bought_unit_amount
    total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
    with open('sample-history.txt', 'a', 1) as fsh:
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
        f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    used_jpy_after = used_jpy * (bought_unit_amount - please_sell_unit_amount) / bought_unit_amount

    avg_jpy = used_jpy / bought_amount
    diff_jpy = now_sell_jpy - avg_jpy
    gained_jpy = diff_jpy * please_sell_amount
    with open('transaction-history.txt', 'a', 1) as fth:
        fth.write(
            f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
            f'-{bought_amount-please_sell_amount:.4f}(0.00%) ETH'
            f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
            f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
        f'-{bought_amount-please_sell_amount:.4f}(0.00%) ETH'
        f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
        f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    return {
        'sample_number': sample_number,
        'buy_count': buy_count,
        'sell_count': sell_count,
        'bought_unit_amount': bought_unit_amount,
        'used_jpy': used_jpy,
        'please_buy_unit_amount': please_buy_unit_amount,
        'please_sell_unit_amount': please_sell_unit_amount,
        'cooling_time': cooling_time,
        'total_gained_jpy': total_gained_jpy
    }

## Start Trading

In [ ]:
import os
import yaml

ROBOT_ID = 'robot-status'

PATH = os.getenv('ROBOT_PATH') or '.'
STATUS_FILE = f'{PATH}/{ROBOT_ID}.yaml'
TMPFILE = f'{PATH}/_tmp'


def read_status():
    if os.path.exists(STATUS_FILE):
        with open(STATUS_FILE, 'r') as f:
            return yaml.load(f, Loader=yaml.FullLoader)
    else:
        return {}


def write_status(status):
    with open(TMPFILE, 'w') as f:
        yaml.dump(status, f)
    os.replace(TMPFILE, STATUS_FILE)

In [ ]:
status = read_status()
sample_number = status.get('sample_number', 0)
buy_count = status.get('buy_count', 0)
sell_count = status.get('sell_count', 0)
bought_unit_amount = status.get('bought_unit_amount', 0)
used_jpy = status.get('used_jpy', 0)
bought_average_jpy = status.get('bought_average_jpy', MAX_UNIT_CC_INIT_TRADE_JPY)
please_buy_unit_amount = status.get('please_buy_unit_amount', 0)
please_sell_unit_amount = status.get('please_sell_unit_amount', 0)
cooling_time = status.get('cooling_time', HAPPY_COOLING_TIME)
total_gained_jpy = status.get('total_gained_jpy', 0)

In [ ]:
def get_sell_unit_amount(bought_unit_amount, diff_jpy):
    A, N = 1.5, 16
    if diff_jpy < 0:
        return 0
    if diff_jpy <= MAX_UNIT_CC_GAIN_JPY:
        return int(bought_unit_amount * (A - 1) / (A ** (1 + N - N * diff_jpy / MAX_UNIT_CC_GAIN_JPY) - 1))
    return bought_unit_amount

with open('sample-history.txt', 'a', 1) as fsh, open('transaction-history.txt', 'a', 1) as fth:
    while True:
        now_ticker = pub.get_ticker('eth_jpy')
        now_sell_jpy = int(now_ticker.get('buy', '0'))
        now_buy_jpy = int(now_ticker.get('sell', '1'))

        bought_amount = TRADE_AMOUNT * bought_unit_amount

        if sample_number % TRADE_RATE == 0 and bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount:
            please_sell_unit_amount = 1
            cooling_time = HAPPY_COOLING_TIME

        if bought_unit_amount > 0:
            sell_unit_amount = get_sell_unit_amount(bought_unit_amount, now_sell_jpy - used_jpy/bought_amount)
            if sell_unit_amount > 0:
                sample_number = 0
                please_sell_unit_amount = sell_unit_amount
                cooling_time = HAPPY_COOLING_TIME

        if now_sell_jpy * bought_amount - used_jpy > MAX_GAIN_JPY:
            sample_number = 0
            please_sell_unit_amount = bought_unit_amount
            cooling_time = HAPPY_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy < used_jpy/bought_amount - MAX_UNIT_CC_LOSS_JPY and used_jpy/MAX_USED_JPY > 90/100:
            please_sell_unit_amount = bought_unit_amount
            cooling_time = SAD_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy < MIN_UNIT_CC_INIT_TRADE_JPY:
            please_sell_unit_amount = bought_unit_amount
            cooling_time = SAD_COOLING_TIME

        if please_sell_unit_amount > 0:
            please_sell_amount = TRADE_AMOUNT * please_sell_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
            )

            bought_amount = TRADE_AMOUNT * bought_unit_amount
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fsh.write(
                f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
                f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )

            used_jpy_after = used_jpy * (bought_unit_amount - please_sell_unit_amount) / bought_unit_amount

            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gained_jpy = diff_jpy * please_sell_amount
            total_gained_jpy += gained_jpy
            fth.write(
                f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
                f'-{bought_amount-please_sell_amount:.4f}({used_jpy_after/MAX_USED_JPY*100:.2f}%) ETH'
                f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
                f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy} JPY in total at {datetime.now()}\n\n'
            )

            send_slack(
                f'Gained {gained_jpy:.2f} JPY. Gained {total_gained_jpy:.2f} JPY in total'
                f'{" by Mock" if isinstance(prv, Mock) else " by Short-Term Robot"}.',
                'Power by https://jhub.name/', 'good' if total_gained_jpy > 0 else 'danger'
            )

            buy_count = 0
            sell_count += 1
            bought_unit_amount -= please_sell_unit_amount
            used_jpy *= bought_unit_amount / (bought_unit_amount + please_sell_unit_amount)
            please_buy_unit_amount = 0
            please_sell_unit_amount = 0
            cooling_time = HAPPY_COOLING_TIME

            if sell_count == MAX_SELL_COUNT or total_gained_jpy < -MAX_TOTAL_LOSS_JPY:
                sell_count = 0
                cooling_time = HAPPY_COOLING_TIME
                total_gained_jpy = 0
                break

            if bought_unit_amount == 0 and total_gained_jpy > MAX_GAIN_JPY << 2 and not isinstance(prv, Mock):
                from unittest.mock import Mock
                prv = Mock()
                send_slack(
                    f'Short-Term Robot is turned to Mock', 'Power by https://jhub.name/', 'good'
                )

            write_status({
                'sample_number': sample_number + 1,
                'buy_count': buy_count,
                'sell_count': sell_count,
                'bought_unit_amount': bought_unit_amount,
                'used_jpy': used_jpy,
                'bought_average_jpy': bought_average_jpy,
                'please_buy_unit_amount': please_buy_unit_amount,
                'please_sell_unit_amount': please_sell_unit_amount,
                'cooling_time': cooling_time,
                'total_gained_jpy': total_gained_jpy,
                'trade_amount': TRADE_AMOUNT
            })

            sample_number += 1
            time.sleep(cooling_time/TEST_RATIO)
            continue

        if bought_unit_amount == 0 and now_buy_jpy > MIN_UNIT_CC_INIT_TRADE_JPY and now_buy_jpy < MAX_UNIT_CC_INIT_TRADE_JPY \
                and used_jpy + now_buy_jpy * TRADE_AMOUNT <= MAX_USED_JPY:
            sample_number = 0
            please_buy_unit_amount = 1

        if bought_unit_amount > 0 and now_buy_jpy < used_jpy/bought_amount - MIN_UNIT_CC_LOSS_JPY:
            buy_unit_amount = (bought_unit_amount + 1) // 2
            while buy_unit_amount > 0 and used_jpy + now_buy_jpy * TRADE_AMOUNT * buy_unit_amount > MAX_USED_JPY:
                buy_unit_amount >>= 1
            if buy_unit_amount > 0:
                sample_number = 0
                please_buy_unit_amount = buy_unit_amount

        if please_buy_unit_amount > 0:
            please_buy_amount = TRADE_AMOUNT * please_buy_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_buy_jpy), amount=str(please_buy_amount), side='buy', order_type='market'
            )

            used_jpy_before = used_jpy

            buy_count += 1
            bought_unit_amount += please_buy_unit_amount
            used_jpy += now_buy_jpy * please_buy_amount
            bought_average_jpy = used_jpy / (TRADE_AMOUNT * bought_unit_amount)
            please_buy_unit_amount = 0

            bought_amount = TRADE_AMOUNT * bought_unit_amount
            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fth.write(
                f'BOUGHT {please_buy_amount:.4f}=-{bought_amount-please_buy_amount:.4f}({used_jpy_before/MAX_USED_JPY*100:.2f}%)'
                f'+{bought_amount:.4f}({used_jpy/MAX_USED_JPY*100:.2f}%) ETH for {now_buy_jpy} JPY:'
                f' SELL {bought_amount:.4f} ETH for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: NO.{buy_count} at {datetime.now()}\n'
                f'GAIN {gain_jpy} JPY: GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
            )
        elif sample_number % TRADE_RATE == 0:
            bought_amount = TRADE_AMOUNT * bought_unit_amount
            sell_for_str = f'{now_sell_jpy}'
            if bought_unit_amount > 0:
                avg_jpy = used_jpy / bought_amount
                diff_jpy = now_sell_jpy - avg_jpy
                sell_for_str = f'{diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f}'
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fth.write(
                f'BUY for {now_buy_jpy} JPY: SELL for {sell_for_str} JPY: at {datetime.now()}\n'
                f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )

        bought_amount = TRADE_AMOUNT * bought_unit_amount
        gain_jpy = now_sell_jpy * bought_amount - used_jpy
        total_gain_jpy = total_gained_jpy + gain_jpy
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
        )

        if sample_number % NOTIFY_RATE == 0:
            bought_amount = TRADE_AMOUNT * bought_unit_amount
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            avg_jpy_str = f'{used_jpy/bought_amount:.2f} / ' if bought_unit_amount > 0 else ''
            send_slack(
                f'Gain {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total by Short-Term Robot'
                f' ({avg_jpy_str}{used_jpy/MAX_USED_JPY*100:.2f}%) at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n'
                f'Buy ETH for {now_buy_jpy} JPY. Support level is {MIN_UNIT_CC_INIT_TRADE_JPY} JPY.',
                'Power by https://jhub.name/', 'good' if total_gain_jpy > 0 else 'danger'
            )

        if sample_number % TRADE_RATE == 0:
            write_status({
                'sample_number': sample_number + 1,
                'buy_count': buy_count,
                'sell_count': sell_count,
                'bought_unit_amount': bought_unit_amount,
                'used_jpy': used_jpy,
                'bought_average_jpy': bought_average_jpy,
                'please_buy_unit_amount': please_buy_unit_amount,
                'please_sell_unit_amount': please_sell_unit_amount,
                'cooling_time': cooling_time,
                'total_gained_jpy': total_gained_jpy,
                'trade_amount': TRADE_AMOUNT
            })

        sample_number += 1
        time.sleep(SAMPLE_INTERVAL / TEST_RATIO)

In [ ]:
sell_all()